# FHC + RHC background subtracted event rate & uncertainties

In [ ]:
# closure test or data study? 
isClosureTest = False 

# using NuWro? 
isNuWro = False


In [ ]:
import sys

sys.path.insert(0, 'backend_functions')

import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb


import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT

import top 
from top import *

import uncertainty_functions 
from uncertainty_functions import *

import xsec_functions 
from xsec_functions import smear_matrix

from ROOT import TH1D, TH2D, TDirectory, TH1F, TH2F, TMatrixD

from selection_functions import *

In [ ]:
import io
import json
import os
from datetime import date

In [ ]:
xvar = 'tksh_angle'
true_var = 'opening_angle'

bins = [-1, -0.5, np.cos(100 * np.pi/180), np.cos(80 * np.pi/180), 0.5, 1]
x_ticks = [-1, -0.5, -0.174, 0.174, 0.5, 1]


xlow = bins[0]
xhigh = bins[-1]

x_label = "cos $\\theta_{ep}$"

In [ ]:
if xvar=='tksh_angle': 
    bincenters = 0.5*(np.array(x_ticks)[1:]+np.array(x_ticks)[:-1])

else: 
    bincenters = 0.5*(np.array(x_ticks+[xhigh])[1:]+np.array(x_ticks+[xhigh])[:-1])
    #bincenters = 0.5*(np.array(bins[:-1]+[xhigh])[1:]+np.array(bins[:-1]+[xhigh])[:-1])

In [ ]:
with open('unfolding/variations/FHCVariations_'+xvar+'.json') as f_fhc:
        fhc_dict = json.load(f_fhc)
        
with open('unfolding/variations/RHCVariations_'+xvar+'.json') as f_rhc:
        rhc_dict = json.load(f_rhc)

In [ ]:
binwidth = []

for x in range(len(bincenters)): 
    binwidth.append(round(abs((x_ticks+[xhigh])[x+1]-(x_ticks+[xhigh])[x])/2, 2))  
   # binwidth.append(round(abs((bins[:-1]+[xhigh])[x+1]-(bins[:-1]+[xhigh])[x])/2, 3))  
    
binwidth

## PPFX

In [ ]:
fig = plt.figure(figsize=(8, 5)) 

for v in fhc_dict['ppfx']: 
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=fhc_dict['evt_rate'],
        histtype='step', color='black', linewidth=2)    

plt.xlim(xlow, xhigh)
plt.title("PPFX VARIATIONS (FHC)", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)

plt.show()

In [ ]:

fig = plt.figure(figsize=(8, 5)) 

for v in rhc_dict['ppfx']: 
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=rhc_dict['evt_rate'],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, xhigh)
plt.title("PPFX VARIATIONS (RHC)", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
ppfx_variations = []

fig = plt.figure(figsize=(8, 5)) 

n = 0

for v_fhc, v_rhc in zip(fhc_dict['ppfx'], rhc_dict['ppfx']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    ppfx_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label='UV')
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC PPFX Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel('Reconstructed '+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ppfx_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_ppfx_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['ppfx'], plot=False, 
                    save=False, xticks=x_ticks, xhigh=xhigh) 

In [ ]:
rhc_ppfx_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['ppfx'], plot=False,  save=False, xticks=x_ticks, xhigh=xhigh) 

In [ ]:
## calculate covariance 

ppfx_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    ppfx_variations, plot=True, xticks=x_ticks, xhigh=xhigh, 
                    axis_label='Reconstructed '+x_label, pot='$7.0 \\times 10^{20}$ POT') 
                    #save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ppfx_") 

In [ ]:
len(bincenters)

In [ ]:
len(fhc_ppfx_dict['fractional_uncertainty'])

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_ppfx_dict['fractional_uncertainty'], linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_ppfx_dict['fractional_uncertainty'], linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=ppfx_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.3)

plt.legend(fontsize=13, frameon=False, ncol=2)
plt.title("PPFX Uncertainty", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ppfx_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## Beamline Geometry 

In [ ]:
beamline_runs = ['HornCurrent', 'xHorn1', 'yHorn1', 'BeamSpotSize', 'xHorn2', 'yHorn2', 'WaterOnHorns', 
                'xBeamShift', 'yBeamShift', 'zTargetPosition']

In [ ]:
fhc_beamline_unisim_cov = {}
rhc_beamline_unisim_cov = {}
beamline_unisim_cov = {}

i = 0

for v_fhc, v_rhc in zip(fhc_dict['beamline'], rhc_dict['beamline']): 
    
    print(beamline_runs[i])
    
    variations = []
    
    comb_v = [a+b for a,b in zip(v_fhc[0],v_rhc[0])]
    comb_v2 = [a+b for a,b in zip(v_fhc[1],v_rhc[1])]
    
    variations.append(comb_v)
    variations.append(comb_v2)
    
    fig = plt.figure(figsize=(7, 7)) 
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    
    ax1.tick_params(axis = 'both', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'both', which = 'major', labelsize = 13)
    
    #ax1.errorbar(bincenters, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
    #            color='black', label="CV", xerr=binwidth, linestyle='None')

    ax1.hist(bincenters, bins, weights=comb_v, linewidth=1.3, 
        histtype='step', color='cornflowerblue', label='UV (+$1\sigma$)')
    
    ax1.hist(bincenters, bins, weights=comb_v2, linewidth=1.3, 
            histtype='step', color='darkgoldenrod', label='UV (-$1\sigma$)')
    
    # Ratios to CV 
    #ax2.hist(bincenters, bins,
    #         weights=[x/y for x,y in zip(comb_v, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
    #         color="cornflowerblue", histtype='step', linewidth=1.3)
    
    #ax2.hist(bincenters, bins,
    #         weights=[x/y for x,y in zip(comb_v2, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
    #         color="darkgoldenrod", histtype='step', linewidth=1.3)
    
    ax1.set_xlim(xlow, xhigh)
    ax1.set_title("FHC+RHC "+beamline_runs[i]+" Variations", fontsize=15)
    ax1.set_xticks(x_ticks)
    
    ax1.set_ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)
    ax1.legend(fontsize=13)
    
    ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

    ax2.set_xlabel('Reconstructed '+x_label, fontsize=14)
    ax2.set_ylabel('UV / CV', fontsize=14)
    ax2.set_xlim(xlow, xhigh)
    ax2.set_ylim(0.94, 1.06)
    ax2.set_xticks(x_ticks)
    
    #plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+beamline_runs[i]+"_"+xvar+".pdf", 
    #        transparent=True, bbox_inches='tight')
    
    plt.show()
    
    # compute unisim covariance
    
    fhc_beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                                                    v_fhc, plot=False)
    
    rhc_beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                                                    v_rhc, plot=False)
    
    beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                                    variations, plot=False)
    

    i += 1

In [ ]:
for k,d in enumerate([fhc_beamline_unisim_cov, rhc_beamline_unisim_cov, beamline_unisim_cov]): 
    
    cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
    for variation in d.keys(): 

        for i in range(len(bins)-1): 
            for j in range(len(bins)-1):

                cov[i][j] += d[variation]['cov'][i][j]
                frac_cov[i][j] += d[variation]['frac_cov'][i][j] 


    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
    if k==0: 
        
        fhc_beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
        
    elif k==1: 
        rhc_beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
    elif k==2: 
        beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 


In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.pcolor(bins, bins, beamline_unisim_dict['cov'], cmap='OrRd', edgecolors='k')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
     
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

cbar.set_label(label='$\\nu^{2}$ / $7.0 \\times 10^{20}$ POT$^{2}$', fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)

plt.title('Covariance Matrix', fontsize=16)
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_cov.pdf", 
#            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.pcolor(bins, bins, beamline_unisim_dict['frac_cov'], cmap='OrRd', edgecolors='k', vmin=0)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
     
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

cbar.set_label(label='$\\nu^{2}$ / $7.0 \\times 10^{20}$ POT$^{2}$', fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)

plt.title('Fractional Covariance Matrix', fontsize=16)
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_frac_cov.pdf", 
#            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.pcolor(bins, bins, beamline_unisim_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
     
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

cbar.set_label(label='$\\nu^{2}$ / $7.0 \\times 10^{20}$ POT$^{2}$', fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)

plt.title('Correlation Matrix', fontsize=16)
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_cor.pdf", 
#            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=beamline_unisim_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_beamline_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_beamline_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.15)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Beamline Uncertainty", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## GENIE ms

In [ ]:
genie_variations = []

fig = plt.figure(figsize=(8, 5)) 

n = 0

for v_fhc, v_rhc in zip(fhc_dict['genie_ms'], rhc_dict['genie_ms']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    genie_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label='UV')
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC GENIE (ms) Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel('Reconstructed '+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_genie_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['genie_ms'], plot=False) 

rhc_genie_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['genie_ms'], plot=False) 

In [ ]:
genie_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                     [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                     genie_variations, xticks=x_ticks, xhigh=xhigh,
                     axis_label='Reconstructed '+x_label, pot='$7.0 \\times 10^{20}$ POT',
                     plot=True)#, save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_")


In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=genie_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_genie_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_genie_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.25)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GENIE (ms) Uncertainty", fontsize=16)


#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## GENIE us

In [ ]:
genie_unisims = ['RPA', 'RPA', 'CCMEC', 'AxFFCCQE', 'VecFFCCQE', 'DecayAngMEC', 'ThetaDelta2Npi', 'ThetaDelta2NRad', 
                          'NormCCCOH', 'NormNCCOH', 'xsr_scc_Fv3', 'xsr_scc_Fa3']

In [ ]:
genie_unisim_cov = {}
fhc_genie_unisim_cov = {}
rhc_genie_unisim_cov = {}


i = 1

for v_fhc, v_rhc in zip(fhc_dict['genie_us'], rhc_dict['genie_us']): 
    
    print(genie_unisims[i])
    
    variations = []
    
    comb_v = [a+b for a,b in zip(v_fhc[0],v_rhc[0])]
    
    if i==1:
        comb_v2 = [a+b for a,b in zip(v_fhc[1],v_rhc[1])]
        variations.append(comb_v)
        variations.append(comb_v2)
    
    else: 
        variations.append(comb_v)
    
    fig = plt.figure(figsize=(7, 7)) 
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    
    ax1.tick_params(axis = 'x', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'x', which = 'major', labelsize = 13)
    
    ax1.tick_params(axis = 'y', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'y', which = 'major', labelsize = 13)
    
    #ax1.errorbar(bincenters, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
    #            color='black', label="CV", xerr=binwidth, linestyle='None')
    
    if i==1: 
        
        ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v, linewidth=1.3, 
        histtype='step', color='cornflowerblue', label='UV (+1$\\sigma$)')
        
        ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v2, linewidth=1.3, 
        histtype='step', color='darkgoldenrod', label='UV (-1$\\sigma$)')
        
        ax1.legend(fontsize=13)
        
    else: 
        ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v, linewidth=1.3, 
        histtype='step', color='cornflowerblue')
    
    # Ratios to CV 
    #ax2.hist(bincenters, bins,
    #         weights=[x/y for x,y in zip(comb_v, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
    #         color="cornflowerblue", histtype='step', linewidth=1.3)
    
    #print([x/y for x,y in zip(comb_v, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])])
    
    #if i==1: 
    #    ax2.hist(bincenters, bins,
    #         weights=[x/y for x,y in zip(comb_v2, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
    #         color="darkgoldenrod", histtype='step', linewidth=1.3)
    
    
    ax1.set_xlim(xlow, xhigh)
    if i==1: 
        ax1.set_title("FHC+RHC "+genie_unisims[i]+" Variations", fontsize=15)
    else: 
        ax1.set_title("FHC+RHC "+genie_unisims[i]+" Variation", fontsize=15)
    ax1.set_xticks(x_ticks)
    
    ax1.set_ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)
    
    ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

    ax2.set_xlabel('Reconstructed '+x_label, fontsize=14)
    ax2.set_ylabel('UV / CV', fontsize=14)
    ax2.set_xlim(xlow, xhigh)
    
    if (genie_unisims[i] == 'CCMEC') or (genie_unisims[i] =='ThetaDelta2NRad') or ('xsr' in genie_unisims[i]): 
        ax2.set_ylim(0.99, 1.01)
    else: 
        ax2.set_ylim(0.94, 1.06)
    
    ax2.set_xticks(x_ticks)
    
    #plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+genie_unisims[i]+"_"+xvar+".pdf", 
    #        transparent=True, bbox_inches='tight')
    
    plt.show()
    
    fhc_genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, 
                                                    fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                                                    v_fhc, plot=False, save=False)
    
    rhc_genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, 
                                                    rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                                                    v_rhc, plot=False, save=False)
    
    genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                     [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                     variations, plot=False, save=False)
    

    i += 1

In [ ]:
for k,d in enumerate([fhc_genie_unisim_cov, rhc_genie_unisim_cov, genie_unisim_cov]): 
    
    cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
    for variation in d.keys(): 

        for i in range(len(bins)-1): 
            for j in range(len(bins)-1):

                cov[i][j] += d[variation]['cov'][i][j]
                frac_cov[i][j] += d[variation]['frac_cov'][i][j] 


    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
    if k==0: 
        fhc_genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
        
    elif k==1: 
        rhc_genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
    elif k==2: 
        genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 


In [ ]:
# covariance

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, genie_unisim_dict['cov'], cmap='OrRd', edgecolors='k')
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)
        
plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xlabel('Reconstructed '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

plt.title('Covariance Matrix', fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_cov.pdf", 
#            transparent=True, bbox_inches='tight')


plt.show()

In [ ]:
# frac covariance

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, genie_unisim_dict['frac_cov'], cmap='OrRd', edgecolors='k', vmin=-0.001, vmax=0.003)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)
        
plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xlabel('Reconstructed '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

plt.title('Fractional Covariance Matrix', fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_frac_cov.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
# correlation

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, genie_unisim_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)
        
plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xlabel('Reconstructed '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

plt.title('Correlation Matrix', fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_cor.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=genie_unisim_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_genie_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_genie_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.25)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GENIE (us) Uncertainty", fontsize=16)


#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## GEANT4

In [ ]:
geant4_variations = []

fig = plt.figure(figsize=(8, 5)) 

n = 0

for v_fhc, v_rhc in zip(fhc_dict['geant4'], rhc_dict['geant4']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    geant4_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label='UV')
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC GEANT4 Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel('Reconstructed '+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/geant4_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()


In [ ]:
fhc_geant4_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['geant4'], plot=False) 

rhc_geant4_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['geant4'], plot=False) 

In [ ]:
geant4_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    geant4_variations, plot=True, xticks=x_ticks, xhigh=xhigh, axis_label="Reconstructed "+x_label, 
                     pot='$7.0 \\times 10^{20}$ POT')
                    #save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/geant4_") 

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=geant4_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_geant4_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_geant4_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GEANT4 Uncertainty", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/geant4_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_geant4_dict['fractional_uncertainty']

## Detector systematics -- Using Reco X

In [ ]:
with open('unfolding/variations/FHCDetSysVariations_July31.json') as f_fhc_detsys:
    fhc_detsys_dict = json.load(f_fhc_detsys)
    
fhc_detsys_dict['LYAttenuation_intrinsic'] = [0 for i in range(len(fhc_detsys_dict['CV_intrinsic'] ))]

with open('unfolding/variations/RHCDetSysVariations_July31.json') as f_rhc_detsys:
    rhc_detsys_dict = json.load(f_rhc_detsys)

In [ ]:
xvar_detsys = "reco_nu_vtx_sce_x"
bins_detsys = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

bincenters_detsys = 0.5*(np.array(bins_detsys)[1:]+np.array(bins_detsys)[:-1])
x_err_detsys = [ round(abs(bins_detsys[x+1]-bins_detsys[x])/2, 3) for x in range(len(bins_detsys)-1) ]

In [ ]:
for variation in intrinsic_detvar_run3_rhc.keys(): 
    
    if variation=='CV_intrinsic': 
        continue
        
    if variation=='LYAttenuation_intrinsic': 
        comb_v = rhc_detsys_dict[variation]
        comb_cv = rhc_detsys_dict['CV_intrinsic']
        
    else: 
        comb_v = [a+b for a,b in zip(fhc_detsys_dict[variation], rhc_detsys_dict[variation])]
        comb_cv = [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])]
        
    fig = plt.figure(figsize=(7, 7)) 
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    
    ax1.tick_params(axis = 'x', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'x', which = 'major', labelsize = 13)
    
    ax1.tick_params(axis = 'y', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'y', which = 'major', labelsize = 13)
        
    ax1.hist(bincenters_detsys, bins_detsys, histtype='step', range=[bins_detsys[0], bins_detsys[-1]], 
                 weights=comb_v, color='cornflowerblue', linewidth=1.3)
    # Ratios to CV 
    
    ratio = [x/y for x,y in zip(comb_v, comb_cv)]
    
    ax2.hist(bincenters_detsys, bins_detsys,
             weights=ratio, 
             color="cornflowerblue", histtype='step', linewidth=1.3)
    
    if variation=='LYAttenuation_intrinsic': 
        ax1.set_title(variation[:-10]+'\nIntrinsic $\\nu_{e}$ CC events (RHC Only)', fontsize=15)
        
    else: 
        ax1.set_title(variation[:-10]+'\nIntrinsic $\\nu_{e}$ CC events', fontsize=15)
    ax1.set_ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)
    ax1.set_xlim(0, 250)
    ax1.set_ylim(0, 25)
    
    
    ax2.set_xlabel("Reconstructed X Position [cm]", fontsize=14)
    ax2.set_ylabel('UV / CV', fontsize=14)
    ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.set_xlim(0, 250)
    ax2.set_ylim(0.9, 1.15)
    
    #plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/detsys_"+variation+"_recox.pdf", 
    #        transparent=True, bbox_inches='tight')
    
    plt.show()

In [ ]:
detsys_cv = [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])]

In [ ]:
detsys_cov_nueCC = {}
fhc_detsys_cov_nueCC = {}
rhc_detsys_cov_nueCC = {}

for variation in intrinsic_detvar_run3_rhc.keys(): 
    
    print(variation)
    
    if variation=='CV': 
        continue
        
    if variation=='LYAttenuation_intrinsic': 
        
        detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          rhc_detsys_dict['CV_intrinsic'], 
                                              rhc_detsys_dict['CV_intrinsic'], 
                                              [rhc_detsys_dict[variation]])
        
        rhc_detsys_cov_nueCC[variation] = detsys_cov_nueCC[variation]
        
    else: 
   
        detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          detsys_cv, detsys_cv, 
                                              [[a+b for a,b in zip(fhc_detsys_dict[variation], rhc_detsys_dict[variation])]])
    
        fhc_detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          fhc_detsys_dict['CV_intrinsic'], 
                                              fhc_detsys_dict['CV_intrinsic'], 
                                              [fhc_detsys_dict[variation]])
        
        rhc_detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          rhc_detsys_dict['CV_intrinsic'], 
                                              rhc_detsys_dict['CV_intrinsic'], 
                                              [rhc_detsys_dict[variation]])
  

In [ ]:
frac_cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]

fhc_frac_cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]
rhc_frac_cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]


for i in range(len(bins_detsys)-1): 
    for j in range(len(bins_detsys)-1):
            
        #cov[i][j] = sum([detsys_cov_nueCC[x]['cov'][i][j] for x in intrinsic_detvar_run3_rhc.keys() if x is not 'CV_intrinsic'])
        frac_cov[i][j] = sum([detsys_cov_nueCC[x]['frac_cov'][i][j] for x in intrinsic_detvar_run3_rhc.keys() if x is not 'CV_intrinsic'])
        
        fhc_frac_cov[i][j] = sum([fhc_detsys_cov_nueCC[x]['frac_cov'][i][j] for x in intrinsic_detvar_run1_fhc.keys() if x is not 'CV_intrinsic'])
        rhc_frac_cov[i][j] = sum([rhc_detsys_cov_nueCC[x]['frac_cov'][i][j] for x in intrinsic_detvar_run3_rhc.keys() if x is not 'CV_intrinsic'])
 


In [ ]:
# now compute absolute total covariance - from frac cov to account for LYAttenuation (RHC only)
cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]

for i in range(len(bins_detsys)-1): 
    for j in range(len(bins_detsys)-1): 
        cov[i][j] = frac_cov[i][j] * detsys_cv[i] * detsys_cv[j]

In [ ]:
cor = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]

# compute the corr matrix 
for i in range(len(cov)): 
    for j in range(len(cov[i])): 

        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
            cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))

nueCC_detsys_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov)),
    'cor' : cor
} 


In [ ]:
# cov

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins_detsys, bins_detsys, nueCC_detsys_dict['cov'], cmap='OrRd', edgecolors='k')
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Reconstructed X [cm]", fontsize=14)

plt.title('Covariance Matrix', fontsize=16)

plt.show()

In [ ]:
# frac cov 

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins_detsys, bins_detsys, nueCC_detsys_dict['frac_cov'], cmap='OrRd', edgecolors='k')
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Reconstructed X [cm]", fontsize=14)

plt.title('Fractional Covariance Matrix', fontsize=16)


plt.show()

In [ ]:
# cor

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins_detsys, bins_detsys, nueCC_detsys_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Reconstructed X [cm]", fontsize=14)

plt.title('Correlation Matrix', fontsize=16)



plt.show()

In [ ]:
plt.figure(figsize=(8, 5))

plt.hist(bincenters_detsys, bins_detsys, weights=np.sqrt(np.diagonal(fhc_frac_cov)), histtype='step', 
         color='limegreen', linewidth=2)

plt.hlines(np.average(np.sqrt(np.diagonal(fhc_frac_cov))), 0, 250, linestyle='--', color='darkgray', linewidth=2,
           label='Average = '+str(round(np.average(np.sqrt(np.diagonal(fhc_frac_cov))), 5)))

plt.xlim(0, 250)
plt.ylim(0, 0.23)
plt.legend(fontsize=13, loc='upper left', frameon=False)

plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("FHC Run 1 Detector Systematics\n$\\nu_{e}$ CC Intrinsic Sample", fontsize=15)


#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/fhc_detsys_recox_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
plt.figure(figsize=(8, 5))

plt.hist(bincenters_detsys, bins_detsys, weights=np.sqrt(np.diagonal(rhc_frac_cov)), histtype='step', 
         color='red', linewidth=2)

plt.hlines(np.average(np.sqrt(np.diagonal(rhc_frac_cov))), 0, 250, linestyle='--', color='darkgray', linewidth=2,
           label='Average = '+str(round(np.average(np.sqrt(np.diagonal(rhc_frac_cov))), 5)))

plt.xlim(0, 250)
plt.ylim(0, 0.3)
plt.legend(fontsize=13, loc='upper left', frameon=False)

plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("RHC Run 3 Detector Systematics\n$\\nu_{e}$ CC Intrinsic Sample", fontsize=15)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/rhc_detsys_recox_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:

x_err = [ round(abs(bins[x+1]-bins[x])/2, 3) for x in range(len(bins)-1) ]

plt.figure(figsize=(8, 5))

#cv_bdtcut = 8767.2+5945.0 # GENIE/PPFX weighted number of events for stat uncertainty 
#cv_bdtcut_err = [np.sqrt(x)/x for x in cv_bdtcut]

plt.hist(bincenters_detsys, bins_detsys, weights=nueCC_detsys_dict['fractional_uncertainty'], histtype='step', 
          color='black', linewidth=2)

#plt.errorbar(bincenters_detsys, nueCC_detsys_dict['fractional_uncertainty'], fmt='none', 
#             yerr=cv_bdtcut_err, color='gray')


plt.hlines(np.average(nueCC_detsys_dict['fractional_uncertainty']), 0, 250, linestyle='--', color='darkgray', linewidth=2,
           label='Average = '+str(round(np.average(nueCC_detsys_dict['fractional_uncertainty']), 5)))

plt.xlim(0, 250)
plt.ylim(0, 0.23)
plt.legend(fontsize=13, loc='upper left', frameon=False)

plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("FHC Run 1 & RHC Run 3 Detector Systematics\n$\\nu_{e}$ CC Intrinsic Sample", fontsize=15)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/detsys_recox_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
avg_detsys_dict = {
    #'fractional_uncertainty' : [np.average(nueCC_detsys_dict['fractional_uncertainty']) for x in range(len(bins))]
}

In [ ]:
avg_detsys_dict['fractional_uncertainty'] = []

for i in range(len(bins)-1):
    if [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])][i] != 0: 
        avg_detsys_dict['fractional_uncertainty'].append(np.average(nueCC_detsys_dict['fractional_uncertainty']))
    
    else: 
        avg_detsys_dict['fractional_uncertainty'].append(0)

In [ ]:
avg_detsys_dict['fractional_uncertainty']

In [ ]:
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
 
for i in range(len(bins)-1): 
    if [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])][i] != 0: 
        frac_cov[i][i] = np.average(nueCC_detsys_dict['fractional_uncertainty'])**2

avg_detsys_dict['frac_cov'] = frac_cov

In [ ]:
avg_detsys_dict['frac_cov']

## Stat uncertainty (number of events)

In [ ]:
# N_data - sumw2_EXT - sumw2_MCbkgd = stat covariance 
# square root the whole thing and divide by evt rate for the fractional uncertainty 

In [ ]:
stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
rhc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_stat_percent_error = []
rhc_stat_percent_error = []
stat_percent_error = []

In [ ]:
isClosureTest

In [ ]:
isNuWro

In [ ]:
if isClosureTest: 
    if isNuWro: 
        nuwro_full_sumw2 = fhc_dict['nuwro_full_sumw2'] #_statlimited']
    mc_signal_sumw2 = [a+b for a,b in zip(fhc_dict['mc_signal_sumw2'], rhc_dict['mc_signal_sumw2'])]
    
else: 
    cv_bkgd = [a+b for a,b in zip(fhc_dict['cv_bkgd'], rhc_dict['cv_bkgd'])]
    beamon_counts = [a+b for a,b in zip(fhc_dict['beamon_counts'], rhc_dict['beamon_counts'])] # data (background-subtracted)
    print(beamon_counts)

mc_bkgd_sumw2 = [a+b for a,b in zip(fhc_dict['mc_bkgd_sumw2'], rhc_dict['mc_bkgd_sumw2'])]
ext_sumw2 = [a+b for a,b in zip(fhc_dict['ext_sumw2'], rhc_dict['ext_sumw2'])]
evt_rate = [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])]


In [ ]:
# need to take the fraction WRT the full event rate (?) 

if isClosureTest: 
    
    for i in range(len(bins)-1):
        
        if isNuWro: 
            
            fhc_frac_stat_cov[i][i] = (nuwro_full_sumw2[i] + fhc_dict['mc_bkgd_sumw2'][i] + fhc_dict['ext_sumw2'][i]) / (fhc_dict['evt_rate_nuwro_ppfx'][i]**2)
            fhc_stat_percent_error.append( np.sqrt( fhc_frac_stat_cov[i][i]) ) 
        
        else: 
            stat_cov[i][i] = (mc_signal_sumw2[i] + mc_bkgd_sumw2[i] + ext_sumw2[i])

            frac_stat_cov[i][i] = stat_cov[i][i] / (evt_rate[i]**2)

            # this is OK -- just take the sum of the weights squared of the full event rate for GENIE closure test 
            fhc_frac_stat_cov[i][i] = ( fhc_dict['mc_signal_sumw2'][i] + fhc_dict['mc_bkgd_sumw2'][i] + fhc_dict['ext_sumw2'][i]) / (fhc_dict['evt_rate'][i]**2)
            rhc_frac_stat_cov[i][i] = ( rhc_dict['mc_signal_sumw2'][i] + rhc_dict['mc_bkgd_sumw2'][i] + rhc_dict['ext_sumw2'][i]) / (rhc_dict['evt_rate'][i]**2)

            fhc_stat_percent_error.append( np.sqrt( fhc_frac_stat_cov[i][i]) ) 
            rhc_stat_percent_error.append(np.sqrt(rhc_frac_stat_cov[i][i])) 
            stat_percent_error.append(np.sqrt(frac_stat_cov[i][i])) 
        
else: 
    
    # need to use the full event rates - before background subtraction
    delta_n_data2 = [a+b for a,b in zip(beamon_counts,cv_bkgd)] 
    delta_n_data2_fhc = [a+b for a,b in zip(fhc_dict['beamon_counts'],fhc_dict['cv_bkgd'])]
    delta_n_data2_rhc = [a+b for a,b in zip(rhc_dict['beamon_counts'],rhc_dict['cv_bkgd'])]
    
    for i in range(len(bins)-1):
    
        stat_cov[i][i] = (delta_n_data2[i] + mc_bkgd_sumw2[i] + ext_sumw2[i])

        if delta_n_data2[i]!=0: 
            frac_stat_cov[i][i] = stat_cov[i][i] / (delta_n_data2[i]**2)

            fhc_frac_stat_cov[i][i] = ( delta_n_data2_fhc[i] + fhc_dict['mc_bkgd_sumw2'][i] + fhc_dict['ext_sumw2'][i]) / (fhc_dict['beamon_counts'][i]**2)
            rhc_frac_stat_cov[i][i] = ( delta_n_data2_rhc[i] + rhc_dict['mc_bkgd_sumw2'][i] + rhc_dict['ext_sumw2'][i]) / (rhc_dict['beamon_counts'][i]**2)

        fhc_stat_percent_error.append( np.sqrt( fhc_frac_stat_cov[i][i]) ) 
        rhc_stat_percent_error.append(np.sqrt(rhc_frac_stat_cov[i][i])) 
        stat_percent_error.append(np.sqrt(frac_stat_cov[i][i]))

    

In [ ]:
#nuwro_full_sumw2 = fhc_dict['nuwro_full_sumw2']
#for i in range(len(bins)-1):
#    fhc_frac_stat_cov[i][i] = (nuwro_full_sumw2[i] + fhc_dict['mc_bkgd_sumw2'][i] + fhc_dict['ext_sumw2'][i]) / (fhc_dict['evt_rate_nuwro_ppfx'][i]**2)
#    fhc_stat_percent_error.append( np.sqrt( fhc_frac_stat_cov[i][i]) ) 



In [ ]:

if not isNuWro: 
    fig = plt.figure(figsize=(8, 5))  


    plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
             weights=stat_percent_error, color='black', linewidth=2)

    plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
             weights=fhc_stat_percent_error, 
             linestyle=(0, (1, 1)), linewidth=2, color='limegreen')

    plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
             weights=rhc_stat_percent_error, 
             linestyle=(0, (1, 1)), color='red', linewidth=2)

    plt.xticks(x_ticks, fontsize=13)
    plt.yticks(fontsize=13)

    plt.xlabel("Reconstructed " + x_label, fontsize=15)
    plt.ylabel("Fractional Uncertainty", fontsize=15)

    plt.xlim(bins[0], xhigh)
    plt.ylim(0, 1.0)

    plt.legend(fontsize=13, frameon=False, ncol=3, loc='upper left')
    plt.title("Statistical Uncertainty on the Event Count", fontsize=16)


    #plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/stat_evt_count_"+xvar+"_frac.pdf", 
    #            transparent=True, bbox_inches='tight')

    plt.show()


## Stat uncertainty (response matrix)

In [ ]:
rmatrix_variations = []

fig = plt.figure(figsize=(8, 5)) 
n = 0

for v_fhc, v_rhc in zip(fhc_dict['response_matrix'], rhc_dict['response_matrix']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    rmatrix_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label="UV")
        
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow,xhigh)
plt.title("FHC+RHC Response Matrix Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel("Reconstructed "+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/rmatrix_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_rmatrix_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['response_matrix'], plot=False, save=False) 



In [ ]:
rhc_rmatrix_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['response_matrix'], plot=False, save=False) 

In [ ]:
rmatrix_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    rmatrix_variations, plot=False, save=False) 

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=rmatrix_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_rmatrix_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_rmatrix_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.12)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Response Matrix Uncertainty", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/rmatrix_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## Dirt Uncertainty 

In [ ]:
dirt_variations = []

fig = plt.figure(figsize=(8, 5)) 


comb_v = [a+b for a,b in zip(fhc_dict['dirt'],rhc_dict['dirt'])]
dirt_variations.append(comb_v)
    
plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1, label='UV')

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['cv_dirt'], rhc_dict['cv_dirt'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC Dirt Variation", fontsize=15)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

plt.legend(fontsize=14, frameon=False)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel('Reconstructed '+x_label, fontsize=14)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/dirt_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fhc_dirt_dict = dirt_unisim(xvar, bins, fhc_dict['evt_rate'], fhc_dict['cv_dirt'], 1.0, isrun3=False, plot=False, 
                               x_label=None, title=None)



In [ ]:
rhc_dirt_dict = dirt_unisim(xvar, bins, rhc_dict['evt_rate'], rhc_dict['cv_dirt'], 1.0, isrun3=True, plot=False, 
                               x_label=None, title=None)

In [ ]:
dirt_dict = dirt_unisim(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                        [a+b for a,b in zip(fhc_dict['cv_dirt'], rhc_dict['cv_dirt'])], 
                        1.0, plot=False, x_label=None, title=None)

In [ ]:
fhc_dirt_dict['fractional_uncertainty']

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=dirt_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_dirt_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_dirt_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(0, 0.025)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Dirt Uncertainty", fontsize=15)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/dirt_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## POT counting 

In [ ]:
pot_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['pot_counting'], rhc_dict['pot_counting']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    pot_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC POT Counting Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
pot_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    pot_variations, plot=False, save=False) 

# Full Uncertainty 

In [ ]:
xvar

### FHC only 

In [ ]:
fhc_frac_cov_dict = {
        'ppfx' : fhc_ppfx_dict['frac_cov'], 
        'beamline' : fhc_beamline_unisim_dict['frac_cov'], 
        'genie_ms' : fhc_genie_dict['frac_cov'], 
        'genie_us': fhc_genie_unisim_dict['frac_cov'], 
        'geant4' : fhc_geant4_dict['frac_cov'],
        'detector' :  [ [0]*(len(bins)-1) for x in range(len(bins)-1) ], 
        'pot_counting' : pot_dict['frac_cov'], #[ [round(0.02**2, 5)]*(len(bins)-1) for x in range(len(bins)-1) ], 
        'dirt' : fhc_dirt_dict['frac_cov'],
        'response_matrix' : fhc_rmatrix_dict['frac_cov'], 
        'stat' : fhc_frac_stat_cov
    }

for i in range(len(bins)-1): 
        fhc_frac_cov_dict['detector'][i][i] = parameters(ISRUN3=False)['detsys_flat']**2


In [ ]:
fhc_tot_frac_cov, fhc_tot_abs_cov = plotFullCov(fhc_frac_cov_dict, xvar, fhc_dict['evt_rate'], 
                                                bins, xlow, xhigh, x_ticks=x_ticks, 
                                                axis_label='Reconstructed '+x_label, pot="2.0 $\\times 10^{20}$ POT")
    

In [ ]:
# add genie in quadrature
fhc_frac_cov_dict['genie_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(fhc_frac_cov_dict['genie_ms'], fhc_frac_cov_dict['genie_us'])]

# add stat in quadrature 
fhc_frac_cov_dict['stat_all'] = [ [x+y for x,y in zip(b,c)] for b,c in zip(fhc_frac_cov_dict['response_matrix'], fhc_frac_cov_dict['stat'])]

# add ppfx & beamline geometry in quadrature
fhc_frac_cov_dict['flux_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(fhc_frac_cov_dict['ppfx'], fhc_frac_cov_dict['beamline'])]


In [ ]:
fhc_frac_unc_dict = {
        'flux_all' : np.sqrt(np.diagonal(fhc_frac_cov_dict['flux_all'])), 
        'genie_all' : np.sqrt(np.diagonal(fhc_frac_cov_dict['genie_all'])), 
        'geant4' : np.sqrt(np.diagonal(fhc_frac_cov_dict['geant4'])),
        'detector' : np.sqrt(np.diagonal(fhc_frac_cov_dict['detector'])), 
        'pot_counting' : np.sqrt(np.diagonal(fhc_frac_cov_dict['pot_counting'])), 
        'dirt' : np.sqrt(np.diagonal(fhc_frac_cov_dict['dirt'])),
        'stat_all' : np.sqrt(np.diagonal(fhc_frac_cov_dict['stat_all'])), 
        'total' : np.sqrt(np.diagonal(fhc_tot_frac_cov))
    }

In [ ]:
fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', weights=np.sqrt(np.diag(fhc_tot_frac_cov)), color='black', 
            linewidth=2, label='Total')

# FLUX
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Flux", 
             weights=fhc_frac_unc_dict['flux_all'])#, color='royalblue')

# CROSS SECTION MODELS 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE", 
             weights=fhc_frac_unc_dict['genie_all'])#, color='goldenrod')

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GEANT4", 
             weights=fhc_frac_unc_dict['geant4'])#, color='green')

# DETECTOR 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Detector", 
             weights=fhc_frac_unc_dict['detector'])#, color='crimson')

# POT COUNTING 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="POT counting",
        weights=fhc_frac_unc_dict['pot_counting'])#, color='purple')

# DIRT 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Dirt",
            weights=fhc_frac_unc_dict['dirt'])#, color='brown')

# STATISTICAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Stat",
            weights=fhc_frac_unc_dict['stat_all'])#, color='hotpink')



plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 1.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("FHC Background-Subtracted Event Rate Uncertainty", fontsize=16)

    #if isNuWro: 
    #    plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/fhc/fakedata/nuwro_nosys_"+xvar+"_FracUncertainty.pdf", 
    #                transparent=True, bbox_inches='tight')


    #    plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/fhc/fakedata/nuwro_"+xvar+"_FracUncertainty.pdf", 
    #                transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_abs_cov_dict = {
        'ppfx' : fhc_ppfx_dict['cov'], 
        'beamline' : fhc_beamline_unisim_dict['cov'], 
        'genie_ms' : fhc_genie_dict['cov'], 
        'genie_us': fhc_genie_unisim_dict['cov'], 
        'geant4' : fhc_geant4_dict['cov'],
        'detector' : [ [0]*(len(bins)-1) for x in range(len(bins)-1) ], 
        'pot_counting' : [ [0]*(len(bins)-1) for x in range(len(bins)-1) ], 
        'dirt' : fhc_dirt_dict['cov'],
        'response_matrix' : fhc_rmatrix_dict['cov'], 
        'stat' : [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

}
    
# convert the detector & stat uncertainty into an absolute covariance (multiply by event rate)
for i in range(len(bins)-1): 
    for j in range(len(bins)-1): 
        
        if i==j: 
            fhc_abs_cov_dict['detector'][i][j] = (0.12187*0.12187) * fhc_dict['evt_rate'][i] * fhc_dict['evt_rate'][j]
        
        fhc_abs_cov_dict['stat'][i][j] = fhc_frac_stat_cov[i][j] * fhc_dict['evt_rate'][i] * fhc_dict['evt_rate'][j]
        fhc_abs_cov_dict['pot_counting'][i][j] = (0.02*0.02) * fhc_dict['evt_rate'][i] * fhc_dict['evt_rate'][j]
        
            


### RHC only 

In [ ]:
rhc_frac_cov_dict = {
        'ppfx' : rhc_ppfx_dict['frac_cov'], 
        'beamline' : rhc_beamline_unisim_dict['frac_cov'], 
        'genie_ms' : rhc_genie_dict['frac_cov'], 
        'genie_us': rhc_genie_unisim_dict['frac_cov'], 
        'geant4' : rhc_geant4_dict['frac_cov'],
        'detector' :  [ [0]*(len(bins)-1) for x in range(len(bins)-1) ],
        'pot_counting' : pot_dict['frac_cov'], #[ [round(0.02**2, 5)]*(len(bins)-1) for x in range(len(bins)-1) ],  
        'dirt' : rhc_dirt_dict['frac_cov'],
        'response_matrix' : rhc_rmatrix_dict['frac_cov'], 
        'stat' : rhc_frac_stat_cov
    }

for i in range(len(bins)-1): 
         rhc_frac_cov_dict['detector'][i][i] = parameters(ISRUN3=True)['detsys_flat']**2



In [ ]:
rhc_tot_frac_cov, rhc_tot_abs_cov = plotFullCov(rhc_frac_cov_dict, xvar, rhc_dict['evt_rate'], 
                                                bins, xlow, xhigh, x_ticks=x_ticks, save=False, 
                                                axis_label='Reconstructed '+x_label)


In [ ]:
# add genie in quadrature
rhc_frac_cov_dict['genie_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(rhc_frac_cov_dict['genie_ms'], rhc_frac_cov_dict['genie_us'])]

# add stat in quadrature 
rhc_frac_cov_dict['stat_all'] = [ [x+y for x,y in zip(b,c)] for b,c in zip(rhc_frac_cov_dict['response_matrix'], rhc_frac_cov_dict['stat'])]

# add ppfx & beamline geometry in quadrature
rhc_frac_cov_dict['flux_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(rhc_frac_cov_dict['ppfx'], rhc_frac_cov_dict['beamline'])]


In [ ]:
rhc_frac_unc_dict = {
        'flux_all' : np.sqrt(np.diagonal(rhc_frac_cov_dict['flux_all'])), 
        'genie_all' : np.sqrt(np.diagonal(rhc_frac_cov_dict['genie_all'])), 
        'geant4' : np.sqrt(np.diagonal(rhc_frac_cov_dict['geant4'])),
        'detector' : np.sqrt(np.diagonal(rhc_frac_cov_dict['detector'])), 
        'pot_counting' : np.sqrt(np.diagonal(rhc_frac_cov_dict['pot_counting'])), 
        'dirt' : np.sqrt(np.diagonal(rhc_frac_cov_dict['dirt'])),
        'stat_all' : np.sqrt(np.diagonal(rhc_frac_cov_dict['stat_all'])), 
        'total' : np.sqrt(np.diagonal(rhc_tot_frac_cov))
    }

In [ ]:
fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', weights=np.sqrt(np.diag(rhc_tot_frac_cov)), color='black', 
            linewidth=2, label='Total')

# FLUX
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Flux", 
             weights=rhc_frac_unc_dict['flux_all'])#, color='royalblue')

# CROSS SECTION MODELS 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE", 
             weights=rhc_frac_unc_dict['genie_all'])#, color='goldenrod')

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GEANT4", 
             weights=rhc_frac_unc_dict['geant4'])#, color='green')

# DETECTOR 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Detector", 
             weights=rhc_frac_unc_dict['detector'])#, color='crimson')

# POT COUNTING 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="POT counting",
            weights=rhc_frac_unc_dict['pot_counting'])#, color='purple')

# DIRT 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Dirt",
            weights=rhc_frac_unc_dict['dirt'])#, color='brown')

# STATISTICAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Stat",
            weights=rhc_frac_unc_dict['stat_all'])#, color='hotpink')

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 1.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("RHC Background-Subtracted Event Rate Uncertainty", fontsize=16)

plt.show()

### FHC + RHC  

In [ ]:
frac_cov_dict = {
        'ppfx' : ppfx_dict['frac_cov'], 
        'beamline' : beamline_unisim_dict['frac_cov'], 
        'genie_ms' : genie_dict['frac_cov'], 
        'genie_us': genie_unisim_dict['frac_cov'], 
        'geant4' : geant4_dict['frac_cov'],
        'detector' : avg_detsys_dict['frac_cov'], 
        'pot_counting' : pot_dict['frac_cov'], 
        'dirt' : dirt_dict['frac_cov'],
        'response_matrix' : rmatrix_dict['frac_cov'], 
        'stat' : frac_stat_cov

}
    
abs_cov_dict = {
        'ppfx' : ppfx_dict['cov'], 
        'beamline' : beamline_unisim_dict['cov'], 
        'genie_ms' : genie_dict['cov'], 
        'genie_us': genie_unisim_dict['cov'], 
        'geant4' : geant4_dict['cov'],
        'detector' : [ [0]*(len(bins)-1) for x in range(len(bins)-1) ], 
        'pot_counting' : pot_dict['cov'], 
        'dirt' : dirt_dict['cov'],
        'response_matrix' : rmatrix_dict['cov'], 
        'stat' : [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

}
    
# convert the detector & stat uncertainty into an absolute covariance (multiply by event rate)
for i in range(len(bins)-1): 
    for j in range(len(bins)-1): 
        abs_cov_dict['detector'][i][j] = avg_detsys_dict['frac_cov'][i][j] * evt_rate[i] * evt_rate[j]
        abs_cov_dict['stat'][i][j] = frac_stat_cov[i][j] * evt_rate[i] * evt_rate[j]
            



In [ ]:
tot_frac_cov, tot_abs_cov = plotFullCov(frac_cov_dict, xvar, evt_rate, bins, xlow, xhigh, x_ticks=x_ticks,
                      axis_label='Reconstructed '+x_label, pot="7.0 $\\times 10^{20}$ POT", 
                                        save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/")

abs_cov_dict['total'] = tot_abs_cov

In [ ]:
## correlation matrix

tot_cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for i in range(len(bincenters)): 
    for j in range(len(bincenters)): 
        tot_cor[i][j] = tot_abs_cov[i][j] / (np.sqrt(tot_abs_cov[i][i])*np.sqrt(tot_abs_cov[j][j]))
        
# CORRELATION MATRIX 

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, tot_cor, cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

plt.xticks(x_ticks, fontsize=12)
plt.yticks(x_ticks, fontsize=12)

plt.xlim(bins[0],xhigh)
plt.ylim(bins[0],xhigh)

plt.title('Correlation Matrix', fontsize=15)

#if isData: 
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/cor_"+xvar+".pdf", 
#                transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
# add ppfx & beamline geometry in quadrature
frac_cov_dict['flux_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['ppfx'], frac_cov_dict['beamline'])]


In [ ]:
# add genie in quadrature
frac_cov_dict['genie_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['genie_ms'], frac_cov_dict['genie_us'])]
frac_cov_dict['stat_all'] = [ [x+y for x,y in zip(b,c)] for b,c in zip(frac_cov_dict['response_matrix'], frac_cov_dict['stat'])]


In [ ]:
# clean away nans
v = np.array(tot_frac_cov)
v[np.isnan(v)] = 0
tot_frac_cov = v

In [ ]:
frac_unc_dict = {
        'flux_all' : np.sqrt(np.diagonal(frac_cov_dict['flux_all'])), 
        'genie_all' : np.sqrt(np.diagonal(frac_cov_dict['genie_all'])), 
        'geant4' : np.sqrt(np.diagonal(frac_cov_dict['geant4'])),
        'detector' : np.sqrt(np.diagonal(frac_cov_dict['detector'])), 
        'pot_counting' : np.sqrt(np.diagonal(frac_cov_dict['pot_counting'])), 
        'dirt' : np.sqrt(np.diagonal(frac_cov_dict['dirt'])),
        'stat_all' : np.sqrt(np.diagonal(frac_cov_dict['stat_all'])), # does not include beam on STAT 
        'total' : np.sqrt(np.diagonal(tot_frac_cov))
}



In [ ]:
fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Total",
            weights=frac_unc_dict['total'], linewidth=2, color='black')

# FLUX
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Flux", 
         weights=frac_unc_dict['flux_all'])#, color='royalblue')

 # CROSS SECTION MODELS 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE", 
             weights=frac_unc_dict['genie_all'])#, color='goldenrod')

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GEANT4", 
             weights=frac_unc_dict['geant4'])#, color='green')

# DETECTOR 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Detector", 
             weights=frac_unc_dict['detector'])#, color='crimson')

# POT COUNTING 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="POT counting",
            weights=frac_unc_dict['pot_counting'])#, color='purple')

# DIRT 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Dirt",
            weights=frac_unc_dict['dirt'])#, color='brown')

# STATISTICAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Stat",
            weights=frac_unc_dict['stat_all'])#, color='hotpink')


plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.7)

plt.legend(fontsize=13, frameon=False, ncol=4, loc='upper left')
plt.title("FHC+RHC Background-Subtracted Event Rate", fontsize=16)


plt.show()

In [ ]:

fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC+RHC",
        weights=frac_unc_dict['total'], linewidth=2, color='black')

# FHC

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=np.sqrt(np.diag(fhc_tot_frac_cov)), 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')

# RHC
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=np.sqrt(np.diag(rhc_tot_frac_cov)), 
         linestyle=(0, (1, 1)), color='red', linewidth=2)


plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 1.3)

plt.legend(fontsize=13, frameon=False, ncol=3, loc='upper left')
plt.title("FHC+RHC Background-Subtracted Event Rate", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/"+xvar+"_FracUncertainty_comparison.pdf", transparent=True, bbox_inches='tight')


plt.show()

## Background subtracted event rate 

In [ ]:
importlib.reload(sf)
from selection_functions import * 

In [ ]:
x_err = []
for x in range(len(bincenters)):
    x_err.append(round(abs((x_ticks+[xhigh][:-1]+[xhigh])[x+1]-(x_ticks+[xhigh][:-1]+[xhigh])[x])/2, 2))

In [ ]:
fig = plt.figure(figsize=(8, 7))

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
    
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
    
ax1.tick_params(axis = 'both', which = 'major', labelsize = 13)
ax2.tick_params(axis = 'both', which = 'major', labelsize = 13)
    
ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

ax1.set_xticks(bins[:-1])
ax2.set_xticks(bins[:-1])

ax1.set_xlim(bins[0], xhigh)
ax2.set_xlim(bins[0], xhigh)


## PREDICTION ## 

fhc_n_genie = ax1.hist(bincenters, bins, histtype='bar', weights=fhc_dict['evt_rate'], color='orange', alpha=0.5,
            label='GENIE: '+str(round(sum(fhc_dict['evt_rate']), 1)))[0]

fhc_genie_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
for i in range(len(bins)-1): 
    fhc_genie_frac_stat_cov[i][i] = fhc_dict['mc_signal_sumw2'][i]/(fhc_n_genie[i]**2)

## CHI2 COMPARISON ##

# include genie stat covariance
fhc_cov_chi2 = np.array(fhc_tot_frac_cov) + np.array(fhc_genie_frac_stat_cov)

# convert to absolute units
for i in range(len(bins)-1): 
     for j in range(len(bins)-1): 
            fhc_cov_chi2[i][j] = fhc_cov_chi2[i][j] * (fhc_n_genie[i] **2)

fhc_inverse_cov_chi2 = np.linalg.inv(fhc_cov_chi2)
fhc_chi2 = 0

for i in range(len(bins)-1): 
    for j in range(len(bins)-1): 
            fhc_chi2 = fhc_chi2  + ( (fhc_n_genie[i]-fhc_dict['beamon_counts'][i])*fhc_inverse_cov_chi2[i][j]*(fhc_n_genie[j]-fhc_dict['beamon_counts'][j]) )


## DATA ## 

ax1.errorbar(bincenters, fhc_dict['beamon_counts'], yerr=np.sqrt(np.diag(fhc_tot_abs_cov)), xerr=x_err, 
             color="black", fmt='o', markersize=3, 
             label='DATA: '+str(int(sum(fhc_dict['beamon_counts'])))+"\n$\\chi^{2}$/n = "+str(round(fhc_chi2, 1))+"/"+str(len(bins)-1))[0]


ax1.step(list(bins)+[0], [0]+list(fhc_n_genie)+[0], 
         color='saddlebrown', linewidth=1, alpha=0.85)



plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)


ax2.set_xlabel(x_label, fontsize=15)

plt.xlim(bins[0], xhigh)
ax1.set_ylim(0, 60)
ax2.errorbar(bincenters, fhc_dict['beamon_counts']/fhc_n_genie, 
             yerr=get_ratio_err(fhc_dict['beamon_counts'], fhc_n_genie), 
             xerr=x_err, color="black", fmt='o')
ax2.set_ylim(0, 2)
ax2.set_ylabel("DATA / GENIE", fontsize=15)
ax2.axhline(1.0, color='black', lw=1, linestyle='--')

ax1.set_title("FHC Background-Subtracted Event Rate", fontsize=15)
ax1.set_ylabel("$\\nu$ / 2.0 $\\times 10^{20}$ POT", fontsize=15)

    
ax1.legend(frameon=False, fontsize=14)


plt.show()
     


In [ ]:
fig = plt.figure(figsize=(8, 7))

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
    
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
    
ax1.tick_params(axis = 'both', which = 'major', labelsize = 13)
ax2.tick_params(axis = 'both', which = 'major', labelsize = 13)
    
ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

ax1.set_xticks(bins[:-1])
ax2.set_xticks(bins[:-1])

ax1.set_xlim(bins[0], xhigh)
ax2.set_xlim(bins[0], xhigh)


############# GENIE PREDICTION #############

rhc_n_genie = ax1.hist(bincenters, bins, histtype='bar', weights=rhc_dict['evt_rate'], color='orange', alpha=0.5,
            label='GENIE: '+str(round(sum(rhc_dict['evt_rate']), 1)))[0]

rhc_genie_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
for i in range(len(bins)-1): 
    rhc_genie_frac_stat_cov[i][i] = rhc_dict['mc_signal_sumw2'][i]/(rhc_n_genie[i]**2)

############# CHI2 COMPARISON #############

# include genie stat covariance
rhc_cov_chi2 = np.array(rhc_tot_frac_cov) + np.array(rhc_genie_frac_stat_cov)

# convert to absolute units
for i in range(len(bins)-1): 
     for j in range(len(bins)-1): 
            rhc_cov_chi2[i][j] = rhc_cov_chi2[i][j] * (rhc_n_genie[i] **2)

rhc_inverse_cov_chi2 = np.linalg.inv(rhc_cov_chi2)
rhc_chi2 = 0

for i in range(len(bins)-1): 
    for j in range(len(bins)-1): 
            rhc_chi2 = rhc_chi2  + ( (rhc_n_genie[i]-rhc_dict['beamon_counts'][i])*rhc_inverse_cov_chi2[i][j]*(rhc_n_genie[j]-rhc_dict['beamon_counts'][j]) )

            
############# DATA RESULT #############

ax1.errorbar(bincenters, rhc_dict['beamon_counts'], yerr=np.sqrt(np.diag(rhc_tot_abs_cov)), xerr=x_err, 
             color="black", fmt='o', markersize=3, 
             label='DATA: '+str(int(sum(rhc_dict['beamon_counts'])))+"\n$\\chi^{2}$/n = "+str(round(rhc_chi2, 1))+"/"+str(len(bins)-1))[0]


ax1.step(list(bins)+[0], [0]+list(rhc_n_genie)+[0], 
         color='saddlebrown', linewidth=1, alpha=0.85)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)


ax2.set_xlabel(x_label, fontsize=15)


ax1.set_ylim(0, 60)

ax2.errorbar(bincenters, rhc_dict['beamon_counts']/rhc_n_genie, 
             yerr=get_ratio_err(rhc_dict['beamon_counts'], rhc_n_genie), 
             xerr=x_err, color="black", fmt='o')
ax2.set_ylim(0, 2)
ax2.set_ylabel("DATA / GENIE", fontsize=15)
ax2.axhline(1.0, color='black', lw=1, linestyle='--')

ax1.set_title("RHC Background-Subtracted Event Rate", fontsize=15)
ax1.set_ylabel("$\\nu$ / 5.0 $\\times 10^{20}$ POT", fontsize=15)

    
ax1.legend(frameon=False, fontsize=14)


plt.show()




In [ ]:
fig = plt.figure(figsize=(8, 7))

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
    
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
    
ax1.tick_params(axis = 'both', which = 'major', labelsize = 13)
ax2.tick_params(axis = 'both', which = 'major', labelsize = 13)
    
ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

ax1.set_xticks(x_ticks)
ax2.set_xticks(x_ticks)

ax1.set_xlim(bins[0], xhigh)
ax2.set_xlim(bins[0], xhigh)


############# GENIE PREDICTION #############

n_genie = ax1.hist(bincenters, bins, histtype='bar', weights=evt_rate, color='orange', alpha=0.5,
            label='GENIE: '+str(round(sum(evt_rate), 1))+"\n$\\chi^{2}$/n = "+str(round(chi2, 1))+"/"+str(len(bins)-1))[0]

genie_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
for i in range(len(bins)-1): 
    genie_frac_stat_cov[i][i] = (fhc_dict['mc_signal_sumw2'][i]+rhc_dict['mc_signal_sumw2'][i])/(n_genie[i]**2)
    
sim_err = [x*y for x, y in zip(n_genie, np.sqrt(np.diag(genie_frac_stat_cov)))]

low_err = [ x-y for x,y in zip(n_genie, sim_err) ]
low_err.insert(0, low_err[0])

high_err = [ x+y for x,y in zip(n_genie, sim_err)]
high_err.insert(0, high_err[0])

ax1.fill_between(bins, low_err, high_err, step="pre",
                    facecolor=(.25, .25, .25, 0), 
                     edgecolor='darkgray', 
                     hatch='.....', 
                     linewidth=0.0, zorder=2, 
                     label='Stat. Uncertainty')

############# CHI2 CALCULATION #############

# include genie stat covariance
cov_chi2 = np.array(tot_frac_cov) + np.array(genie_frac_stat_cov)

# convert to absolute units
for i in range(len(bins)-1): 
     for j in range(len(bins)-1): 
            cov_chi2[i][j] = cov_chi2[i][j] * (n_genie[i] **2)

inverse_cov_chi2 = np.linalg.inv(cov_chi2)
chi2 = 0

for i in range(len(bins)-1): 
    for j in range(len(bins)-1): 
            chi2 = chi2  + ( (n_genie[i]-beamon_counts[i])*inverse_cov_chi2[i][j]*(n_genie[j]-beamon_counts[j]) )

    
############# DATA RESULT #############

ax1.errorbar(bincenters, beamon_counts, yerr=np.sqrt(np.diag(tot_abs_cov)), xerr=x_err, 
             color="black", fmt='o', markersize=3, 
             label='DATA: '+str(int(sum(beamon_counts))))[0]

ax1.step(list(bins)+[0], [0]+list(n_genie)+[0], 
         color='saddlebrown', linewidth=1, alpha=0.85)


########## RATIO ###############################

ax2.errorbar(bincenters, [x/y for x,y in zip(beamon_counts,n_genie)], 
             yerr=get_ratio_err(beamon_counts, n_genie), 
             xerr=x_err, color="black", fmt='o')

low_err_ratio = [ 1 - x for x in np.sqrt(np.diag(genie_frac_stat_cov)) ]
low_err_ratio.insert(0, low_err_ratio[0])
    
high_err_ratio = [ 1 + x for x in np.sqrt(np.diag(genie_frac_stat_cov)) ]
high_err_ratio.insert(0, high_err_ratio[0])

ax2.fill_between(bins, low_err_ratio, high_err_ratio, step="pre", facecolor=(.25, .25, .25, 0), 
                     edgecolor='darkgray',
                     hatch='.....', 
                     linewidth=0.0, zorder=1)


######### FORMATTING ############################
plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

ax2.set_xlabel('Reconstructed '+x_label, fontsize=15)

ax1.set_ylim(0, 65)

ax2.set_ylim(0, 2)
ax2.set_ylabel("DATA / GENIE", fontsize=15)
ax2.axhline(1.0, color='black', lw=1, linestyle='--')

ax1.set_title("FHC+RHC Background-Subtracted Event Rate", fontsize=15)
ax1.set_ylabel("$\\nu$ / 7.0 $\\times 10^{20}$ POT", fontsize=15)
   
ax1.legend(frameon=False, fontsize=14)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/"+xvar+"_evtrate_sub.pdf", bbox_inches='tight')#, dpi=1000) 
plt.show()



## Smearing & efficiency 

In [ ]:
with open('unfolding/smearing/FHC_'+xvar+'.json') as f_fhc:
        fhc_smearing_dict = json.load(f_fhc)

with open('unfolding/smearing/RHC_'+xvar+'.json') as f_rhc:
        rhc_smearing_dict = json.load(f_rhc)
    

In [ ]:
xvar

In [ ]:
fhc_true_generated_intrinsic = generated_signal(False, true_var, bins, xlow, xhigh, weight='totweight_intrinsic')[0]
rhc_true_generated_intrinsic = generated_signal(True, true_var, bins, xlow, xhigh, weight='totweight_intrinsic')[0]
true_generated_intrinsic = [a+b for a,b in zip(fhc_true_generated_intrinsic,rhc_true_generated_intrinsic)]

In [ ]:
###########################################################################################

# fhc efficiency 
fhc_true_generated = fhc_smearing_dict['true_generated']
fhc_true_selected = fhc_smearing_dict['true_selected']

fhc_eff = [a/b for a,b in zip(fhc_true_selected, fhc_true_generated)]
fhc_eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(fhc_eff,fhc_true_generated_intrinsic) ]

print('FHC Efficiency (%) = ', [round(x,3)*100 for x in fhc_eff])


fig = plt.figure(figsize=(8, 5))  

plt.errorbar(bincenters, fhc_eff, yerr=fhc_eff_err, xerr=binwidth, fmt='.', color='seagreen')

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.ylim(0, .25)

plt.xlabel("True " + x_label, fontsize=15)
plt.ylabel("Selected / Generated", fontsize=15)

plt.xlim(bins[0], xhigh)

plt.title("Efficiency", fontsize=16)

plt.grid(linestyle=':')

plt.show()

In [ ]:


###########################################################################################

# rhc efficiency 
rhc_true_generated = rhc_smearing_dict['true_generated']
rhc_true_selected = rhc_smearing_dict['true_selected']

rhc_eff = [a/b for a,b in zip(rhc_true_selected, rhc_true_generated)]
rhc_eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(rhc_eff,rhc_true_generated_intrinsic) ]

print('RHC Efficiency (%) = ', [round(x,3)*100 for x in rhc_eff])

###########################################################################################

# combined efficiency 

true_generated = [a+b for a,b in zip(fhc_smearing_dict['true_generated'], rhc_smearing_dict['true_generated'])]
true_selected = [a+b for a,b in zip(fhc_smearing_dict['true_selected'], rhc_smearing_dict['true_selected'])]

eff = [a/b for a,b in zip(true_selected, true_generated)]
eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(eff,true_generated_intrinsic) ]

print('Combined Efficiency (%) = ', [round(x,3)*100 for x in eff])


# Plot 

fig = plt.figure(figsize=(8, 5))  

plt.errorbar(bincenters, fhc_eff, yerr=fhc_eff_err, xerr=binwidth, label="FHC", fmt='.', color='limegreen', linewidth=1)
plt.errorbar(bincenters, rhc_eff, yerr=rhc_eff_err, xerr=binwidth, label="RHC", fmt='.', color='red', linewidth=1)

plt.errorbar(bincenters, eff, yerr=eff_err, xerr=binwidth, label="FHC+RHC", fmt='.', color='black', linewidth=1)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.ylim(0, .25)

plt.xlabel("True " + x_label, fontsize=15)
plt.ylabel("Selected / Generated", fontsize=15)

plt.xlim(bins[0], xhigh)

plt.legend(fontsize=14, frameon=False, ncol=3)
plt.title("Efficiency", fontsize=16)

plt.grid(linestyle=':')

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/"+xvar+"_efficiency_comparison.pdf", 
#            transparent=True, bbox_inches='tight')


plt.show()

In [ ]:
response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
smearing_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
rhc_response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for col in range(len(bins)-1):
    for row in range(len(bins)-1):
        
        fhc_response_matrix[row][col] = fhc_smearing_dict['smearing_matrix'][row][col] / fhc_smearing_dict['true_generated'][col]

        if not isNuWro: 
            smearing_matrix[row][col] = (fhc_smearing_dict['smearing_matrix'][row][col] + rhc_smearing_dict['smearing_matrix'][row][col])
            response_matrix[row][col] =  smearing_matrix[row][col] / true_generated[col]

            rhc_response_matrix[row][col] = rhc_smearing_dict['smearing_matrix'][row][col] / rhc_smearing_dict['true_generated'][col]


In [ ]:
fig = plt.figure(figsize=(13, 9))

plt.pcolor(bins, bins, smearing_matrix, cmap='OrRd', vmin=0.00000000001, edgecolors='k')

plt.title('FHC+RHC Smearing', fontsize=15)

plt.xticks(x_ticks, fontsize=14)
plt.gca().xaxis.tick_bottom()
plt.yticks(x_ticks, fontsize=14)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xlabel('True '+x_label, fontsize=15)
plt.ylabel('Reco '+x_label, fontsize=15)

plt.show()

In [ ]:
bincenters

In [ ]:
fig = plt.figure(figsize=(13, 9))

plt.pcolor(bins, bins, response_matrix, cmap='OrRd', vmin=0.0, edgecolors='k')

plt.title('FHC+RHC Response Matrix (Smearing & Efficiency)', fontsize=15)

plt.xticks(x_ticks[1:], fontsize=14)
plt.gca().xaxis.tick_bottom()

plt.yticks(x_ticks[1:], fontsize=14)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

cbar.set_label(label="$\\nu$ / 7.0 $\\times 10^{20}$ POT", fontsize=15)

plt.xlabel('True '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

for i in range(len(bins)-1): # reco bins i (y axis)
    for j in range(len(bins)-1): # true bins j (x axis)

        if not np.isnan(response_matrix[i][j]) and round(response_matrix[i][j],3)>0 :
            plt.text(bincenters[j], bincenters[i], round(response_matrix[i][j], 3), 
                        color='black', ha="center", va="center", fontsize=12)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/"+xvar+"_rmatrix.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
# check that this is constructed properly 
# sum of the columns should equal the combined efficiency 

for i in range(len(bincenters)): # loop over columns

    x = 0 
    for j in range(len(bincenters)): # sum up the rows
    
        x += response_matrix[j][i]

    print(x)

## save the input covariance breakdown 

In [ ]:
if isClosureTest and isNuWro==False:

    cov_path = 'unfolding/covariance/closuretests/'
    #cov_file = xvar+"_Combined_abs_inputcov_ClosureTest.json"

    #if os.path.exists(cov_path+cov_file): 
    #    print(cov_path+cov_file, "exists and is readable, need to update file name to save ! ")

    #else:       
    #    with open(cov_path+cov_file, 'w') as f:
    #            json.dump(abs_cov_dict, f)
                
                
    cov_file_fhc = xvar+"_FHC_abs_inputcov_ClosureTest.json"
    
    if os.path.exists(cov_path+cov_file_fhc): 
        print(cov_path+cov_file_fhc, "exists and is readable, need to update file name to save ! ")
        
    else:       
        with open(cov_path+cov_file_fhc, 'w') as f_fhc:
                json.dump(fhc_abs_cov_dict, f_fhc)

In [ ]:
if isNuWro==True:

    cov_path = 'unfolding/covariance/fakedata/'
    cov_file = xvar+"_FHC_abs_inputcov_NuWro.json"

    if os.path.exists(cov_path+cov_file): 
        print(cov_path+cov_file, "exists and is readable, need to update file name to save ! ")

    else:       
        with open(cov_path+cov_file, 'w') as f:
                json.dump(fhc_abs_cov_dict, f)

## save to unfolding file 

In [ ]:
save = True
path = "/uboone/data/users/kmiller/uBNuMI_CCNp/unfolding/closuretests/"

In [ ]:
xvar

In [ ]:
## check 

check = [0]*(len(bins)-1)
    

for key in fhc_abs_cov_dict.keys(): 
    
    diag = np.diagonal(fhc_abs_cov_dict[key])
    check = [a+b for a,b in zip(check, diag)]
    


In [ ]:
np.array(np.round(check, 1))==np.round(np.diagonal(fhc_tot_abs_cov), 1)

In [ ]:
########### true signal distribution ###########
fhc_htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    
    # use genie true generated as input for both closure tests and fake data studies
    fhc_htrue_signal.Fill(bincenters[i], fhc_smearing_dict['true_generated'][i])
        
        
########### reco distribution (MC or data) ###########
fhc_hreco = TH1F("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))
for i in range(len(bincenters)): 
    
    if isNuWro: 
        fhc_hreco.Fill(bincenters[i], fhc_dict['evt_rate_nuwro_ppfx'][i]) 
        print("Filling reco dist. with NuWro counts....")
    elif isClosureTest: 
        fhc_hreco.Fill(bincenters[i], fhc_dict['evt_rate'][i])
        print("Filling reco dist. with GENIE counts....")
    else: # real data! 
        print("Filling reco dist. with real data counts....")
        fhc_hreco.Fill(bincenters[i], fhc_dict['beamon_counts'][i])


########### covariance matrix ###########
fhc_hcov = TH2D("hcov_tot", "Covariance Matrix of Reco "+x_label, 
                len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))


for i in range(len(bincenters)): # i = row (y)
    for j in range(len(bincenters)): # j = column (x) 

        fhc_hcov.Fill(bincenters[j], bincenters[i], fhc_tot_abs_cov[i][j])

    
########### response matrix ###########
fhc_r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
for i in range(len(bincenters)): # i = column (x)
    for j in range(len(bincenters)): # j = row (y) 
            
        fhc_r.Fill(bincenters[i], bincenters[j], fhc_response_matrix[i][j]) 
            

    

In [ ]:
fhc_filename = "WSVD_"+xvar+"_FHC_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+fhc_filename): 
    print(path+fhc_filename, "exists and is readable, need to update file name to save ! ")
        
else: 
            
    fhc_f = ROOT.TFile.Open(path+fhc_filename, "NEW")

    fhc_f.cd()

    fhc_hcov.Write()
    fhc_htrue_signal.Write()
    fhc_hreco.Write()
    fhc_r.Write()

    ## include the individual covariances 

    for key in fhc_abs_cov_dict.keys(): 

        h = TH2D(key, "Covariance Matrix of Reco "+x_label + " - "+key, 
                    len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
        
        for i in range(len(bincenters)): # i = row (y)
            for j in range(len(bincenters)): # j = column (x) 
                h.Fill(bincenters[j], bincenters[i], fhc_abs_cov_dict[key][i][j])
        h.Write()

    fhc_f.Close()

In [ ]:
fhc_abs_cov_dict.keys()

In [ ]:
# covariance matrix 
rhc_hcov = TH2D("hcov_tot", "Covariance Matrix vs. Reco "+x_label, 
            len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

for i in range(len(bincenters)): # i = row (y)
    for j in range(len(bincenters)): # j = column (x) 

        rhc_hcov.Fill(bincenters[j], bincenters[i], rhc_tot_abs_cov[i][j]) 

# true signal distribution
rhc_htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    rhc_htrue_signal.Fill(bincenters[i], rhc_smearing_dict['true_generated'][i])
        
# reco distribution (MC or data)
rhc_hreco = TH1D("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    
    if isClosureTest: 
        rhc_hreco.Fill(bincenters[i], rhc_dict['evt_rate'][i])
    else: # real data! 
        print("Filling reco dist. with real data counts....")
        rhc_hreco.Fill(bincenters[i], rhc_dict['beamon_counts'][i])
    
# response matrix 
rhc_r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

for i in range(len(bincenters)): # i = column (x)
    for j in range(len(bincenters)): # j = row (y) 
            
        rhc_r.Fill(bincenters[i], bincenters[j], rhc_response_matrix[i][j]) 
            





In [ ]:
rhc_filename = "WSVD_"+xvar+"_RHC_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+rhc_filename): 
    print(rhc_filename, " exists and is readable, need to update file name to save ! ")
        
else: 
            
    rhc_f = ROOT.TFile.Open(path+rhc_filename, "RECREATE")

    rhc_f.cd()
    rhc_hcov.Write()
    rhc_htrue_signal.Write()
    rhc_hreco.Write()
    rhc_r.Write()

    rhc_f.Close()

In [ ]:
# covariance matrix 
hcov = TH2D("hcov_tot", "Covariance Matrix vs. Reco "+x_label, 
            len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

for i in range(len(bincenters)): # i = row (y)
    for j in range(len(bincenters)): # j = column (x) 

        hcov.Fill(bincenters[j], bincenters[i], tot_abs_cov[i][j]) 

# true signal distribution
htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    htrue_signal.Fill(bincenters[i], true_generated[i])
        
# reco distribution (MC or data)
hreco = TH1D("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    if isClosureTest: 
        hreco.Fill(bincenters[i], evt_rate[i])
    else: 
        print("Filling reco dist. with real data counts....")
        hreco.Fill(bincenters[i], beamon_counts[i])

    
# response matrix 
r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
for i in range(len(bincenters)): # i = column (x)
    for j in range(len(bincenters)): # j = row (y) 
            
        r.Fill(bincenters[i], bincenters[j], response_matrix[i][j]) 
            
            
filename = "WSVD_"+xvar+"_Combined_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+filename): 
    print(filename, " exists and is readable, need to update file name to save ! ")
        
else: 
            
    f = ROOT.TFile.Open(path+filename, "RECREATE")

    f.cd()

    hcov.Write()
    htrue_signal.Write()
    hreco.Write()
    r.Write()

    f.Close()            


In [ ]:
beamon_counts

In [ ]:
fhc_r

In [ ]:
data_signal = TMatrixD(len(bins)-1,1)

for i in range(len(bins)-1):
    data_signal[i][0] = rhc_dict['beamon_counts'][i]

data_covmat = TMatrixD(len(bins)-1, len(bins)-1)

for i in range(len(bins)-1):
    for j in range(len(bins)-1):
        data_covmat[i][j] = rhc_tot_abs_cov[i][j]


smearcept = TMatrixD(len(bins)-1,len(bins)-1)

for i in range(len(bins)-1):
    for j in range(len(bins)-1):
        smearcept[i][j] = rhc_response_matrix[i][j]

prior_true_signal = TMatrixD(len(bins)-1, 1)

for i in range(len(bins)-1):
    prior_true_signal[i][0] = rhc_smearing_dict['true_generated'][i]
    

# save to file 

filename = "gardiner_WSVD_"+xvar+"_RHC_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+filename): 
    print(filename, " exists and is readable, need to update file name to save ! ")
        
else: 
            
    f = ROOT.TFile.Open(path+filename, "RECREATE")
    f.cd()

    data_signal.Write("data_signal")
    data_covmat.Write("data_covmat")
    smearcept.Write("smearcept")
    prior_true_signal.Write("prior_true_signal")

    f.Close() 

In [ ]:
path


In [ ]:
hreco

In [ ]:
np.sum(beamon_counts)/np.sum(evt_rate)

## compare input vs. output covariance

In [ ]:
# get the rotation matrix 

with open('unfolding/results/closuretests/CombinedResults_true_e_visible_112822.json') as f:
        results_dict = json.load(f)

In [ ]:
results_dict.keys()

In [ ]:
covrot = np.array(results_dict['covrotation'])

# also need the transpose! 
covrot_t = covrot.transpose()

In [ ]:
# rotate the input covariance 
unfcov = np.matmul( np.matmul(covrot, tot_abs_cov), covrot_t )

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, unfcov, cmap='OrRd', edgecolors='k')#, vmin=650, vmax=750)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel("Unfolded "+x_label, fontsize=15)
plt.ylabel("Unfolded "+x_label, fontsize=15)

plt.xticks(x_ticks, fontsize=12)
plt.yticks(x_ticks, fontsize=12)

plt.xlim(bins[0],xhigh)
plt.ylim(bins[0],xhigh)

plt.title('CovRot * InputCov * CovRot_t', fontsize=15)

plt.show()

In [ ]:
## compute the fractional covariance & uncertainty

unfcov_frac = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for i in range(len(bins)-1):
    for j in range(len(bins)-1): 
        unfcov_frac[i][j] = unfcov[i][j]/(evt_rate[i]*evt_rate[j])

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, unfcov_frac, cmap='OrRd', edgecolors='k')#, vmin=650, vmax=750)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel("Unfolded "+x_label, fontsize=15)
plt.ylabel("Unfolded "+x_label, fontsize=15)

plt.xticks(x_ticks, fontsize=12)
plt.yticks(x_ticks, fontsize=12)

plt.xlim(bins[0],xhigh)
plt.ylim(bins[0],xhigh)

plt.title('Fractional Covariance', fontsize=15)

plt.show()